# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,-1.31,82,100,6.02,SJ,1733406402
1,1,avarua,-21.2078,-159.7750,24.03,83,20,3.09,CK,1733406403
2,2,baardheere,2.3446,42.2764,35.79,20,1,4.93,SO,1733406404
3,3,port-aux-francais,-49.3500,70.2167,6.08,97,92,3.23,TF,1733406405
4,4,stanley,54.8680,-1.6985,9.18,82,20,1.03,GB,1733406408


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Create an interactive map with points for each city, size based on humidity
plot_map = city_data_df.hvplot.points(
    "Lng",  # Longitude on the x-axis
    "Lat",  # Latitude on the y-axis
    geo=True,  # Use geo-coordinates for map plot
    tiles="OSM",  # Set background tiles to OpenStreetMap
    color="City",  # Color by city name (or another column)
    alpha=0.5,  # Set transparency of points
    size="Humidity",  # Size of the points based on humidity
    scale=0.7,  # Scale factor for size
    frame_width=900,  # Set width of the frame for better visibility
    frame_height=600  # Set height of the frame for better visibility
)

# Display the plot
plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter the DataFrame for ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 18) & (city_data_df["Max Temp"] < 33) &  # Temperature range
    (city_data_df["Wind Speed"] < 4.5) &   # Wind speed condition
    (city_data_df["Cloudiness"] == 20)  # 20 cloudiness condition
]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,avarua,-21.2078,-159.7750,24.03,83,20,3.09,CK,1733406403
52,52,acapulco de juarez,16.8634,-99.8901,22.90,83,20,2.06,MX,1733406311
154,154,male,4.1748,73.5089,28.97,74,20,2.57,MV,1733406587
168,168,guasdualito,7.2424,-70.7324,25.98,89,20,1.54,VE,1733406604
311,311,pisco,-13.7000,-76.2167,20.03,83,20,3.09,PE,1733406776


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Display the DataFrame 
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,avarua,CK,-21.2078,-159.7750,83,
52,acapulco de juarez,MX,16.8634,-99.8901,83,
154,male,MV,4.1748,73.5089,74,
168,guasdualito,VE,7.2424,-70.7324,89,
311,pisco,PE,-13.7000,-76.2167,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000  

# Set up the parameters for the Geoapify API request
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "apiKey": "geoapify_key",   # Replace with your Geoapify API key
    "limit": 1  # Return only the closest result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"  # Filter by radius around the coordinates
    params["bias"] = f"proximity:{lng},{lat}"  # Bias results toward this location

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        # Extract the name of the first result
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
avarua - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: No hotel found
male - nearest hotel: No hotel found
guasdualito - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
kuala terengganu - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
1,avarua,CK,-21.2078,-159.7750,83,No hotel found
52,acapulco de juarez,MX,16.8634,-99.8901,83,No hotel found
154,male,MV,4.1748,73.5089,74,No hotel found
168,guasdualito,VE,7.2424,-70.7324,89,No hotel found
311,pisco,PE,-13.7000,-76.2167,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
Hotel_plot_map = hotel_df.hvplot.points(
    "Lng", "Lat",  # Longitude and Latitude for positioning points
    geo=True,      # Enables geographical plotting
    tiles="OSM",   # Background map style (OpenStreetMap)
    frame_width=900, frame_height=600,  # Set the map size
    scale=0.8,        # Scaling factor for size
    color="Country",  # Points color based on the country
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]
)

# Display the plot
Hotel_plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,City,Hotel Name,Humidity)